In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
def scrape_data(url):

  data = {"Game Title" : [],
          "Game Genre" : [],
          "Pricing" : [],
          "Game Library Size" : 'n/a',
          "Publisher" : [],
          "Release Date" : [],
          "Platform" : 'n/a',
          "Rating" : []
}
  try:
    response = requests.get(url)
  except Exception as e:
    raise ValueError(e)

  soup = bs(response.text, 'html.parser')


  # review
  # insufficient reviews will have 'not_enough_reviews', omit the game
  review = soup.find("span", {"class" : "game_review_summary"})
  if "not_enough_reviews" in review.get("class", []):
    raise ValueError("Insufficient reviews")
  data["Rating"] = review.text.strip()

  # price
  # omit demo, early access, etc. by letting it throw an exception error and skip the game
  price = soup.find("div", {"class" : "game_purchase_price price"}) or soup.find("div", {"class" : "discount_original_price"})
  data["Pricing"] = price.text.strip()

  # release date
  data["Release Date"] = soup.find("div", {"class" : "date"}).text.strip().replace(',','')

  # genre
  data["Game Genre"] = soup.find("span", {"data-panel" :'{"flow-children":"row"}'}).text.strip()

  # publisher
  data["Publisher"] = soup.find("div", {"id" : "developers_list", "class" : "summary column"}).text.strip()

  # title
  data["Game Title"] = soup.find("b", string="Title:").next_sibling.strip().strip()


  return pd.DataFrame([data])


In [ ]:
with open("./data/game_urls_filtered.txt", "r") as f:
  urls = f.read().splitlines()

print(len(urls))

"""
Tổng: 154984 link
Mỗi đứa chạy: 51661 link
"""
range1 = range(0, 51661)
range2 = range(51662, 103322)
range3 = range(103323, 154984)


def run(r, index):

  datalist = []

  for i in r:
    try:
      datalist.append(scrape_data(urls[i]))
    except Exception as e:
      continue

  dataset =  pd.concat(datalist)

  print(dataset.shape)
  dataset.head()

  dataset.to_csv(f"part_{index}_steam_data.csv", index=False)

In [ ]:
run(range2, 2)